This training run is done on the new dataset with 1830 airfoils, 1 mach #, 25 AoA #s, and 3 Re #s.

It runs on the redefined NIF model.

In [11]:
%load_ext autoreload
%autoreload 2

# Define autroreload so that it doesn't cause pain in the ass when we change the functions and run this notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Import all the models, training functions, manipulators here

# Define the relative paths, append it to the system path
import sys
from pathlib import Path
project_root = Path.cwd().resolve().parents[4]
github_root = Path.cwd().resolve().parents[3]
sys.path.append(str(project_root))
sys.path.append(str(github_root))

print(project_root)
print(github_root)

# Import shenanigans
from defs.helper_functions.training_functions import *
from defs.helper_functions.data_loaders import *
from defs.models.NIF import *

# Time, to precisely: time
import time

C:\SenkDosya\Projects\AeroML
C:\SenkDosya\Projects\AeroML\initial-project


In [13]:
# Set device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")  # Force CPU for testing purposes
print(f"Using device: {device}")

Using device: cuda


In [14]:
# Figure out the data
df = pd.read_csv(rf"C:\SenkDosya\Projects\AeroML\airfoil_data\airfoil_dataset_8bern.csv")

df = df.drop(['N1', 'N2'], axis=1) # Remove N1 and N2 since all the airfoils are subsonic

df['Reynolds'] = df['Reynolds'] / 100000

geom, cond, perf, names= get_dataset(df, loc_geometry=[1,20], loc_cond=[21,23], loc_perf_coeffs=[24,26], loc_names=0) # Get the necessary stuff for the dataset
print(df.shape); print(geom.shape); print(cond.shape); print(perf.shape); print(len(names))

ds = AirfoilDataset(geom, cond, perf, names)

del df, geom, cond, perf, names # Delete these to preserve memory

cfg_loader = {
    'n_epoch': 100,
    'n_train': 1000,
    'n_test': 17250,
    'train_batch': 1
}

dl_train, dl_val, dl_test = get_dataloaders(ds=ds, cfg_loader=cfg_loader)


(137250, 27)
torch.Size([137250, 20])
torch.Size([137250, 3])
torch.Size([137250, 3])
137250


In [15]:
import torch.nn as nn

# Define the model and optimizer

cfg_shape_net = {
    'input_dim': 20,
    'output_dim': 3,
    'hidden_units': [2048,2048,2048,2048,2048],
    'shape_activation': nn.SELU
}

cfg_param_net = {
    'input_dim': 3,
    'hidden_units': [512,512,512],
    'param_activation': nn.Tanh
}

model = NIF_Pointwise(cfg_shape_net=cfg_shape_net, cfg_param_net=cfg_param_net)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.01)

model = model.to(device)

# Define the loss_fn

loss_fn = nn.MSELoss()

In [16]:
# Show the model

from torchviz import make_dot

# Create and move inputs to device
show_tensor_1 = torch.randn(5, cfg_shape_net['input_dim']).to(device)
show_tensor_2 = torch.randn(5, cfg_param_net['input_dim']).to(device)

# Forward pass
output = model(show_tensor_1, show_tensor_2)

make_dot(output, params=dict(model.named_parameters())).render("test", format="png")

'test.png'

In [17]:
# Train the model

# Define the configs of train
cfg_train = {
    'cfg_loader': cfg_loader,
    'dtype': torch.float32,
    'device': device
}

iter_train = iter(dl_train)

dtype = torch.float32

batch= next(iter_train)

for _ in range(100):
    model.train() # Set the model to training mode

    shape_input, param_input, target, airfoil_name = batch['geometry'].to(device= device, dtype= dtype), batch['cond'].to(device= device, dtype= dtype), batch['perf_coeffs'].to(device= device, dtype= dtype), batch['name'] 

    optimizer.zero_grad()

    # Get the predictions
    pred = model(shape_input= shape_input, param_input= param_input)

    # Get the loss 
    loss = loss_fn(target, pred)
    
    # Backprop
    loss.backward()
    optimizer.step()

    print(f'Train run {_+1} loss: {loss}')
    


Train run 1 loss: 0.7417542338371277
Train run 2 loss: 0.7416312098503113
Train run 3 loss: 0.7415128946304321
Train run 4 loss: 0.7413613200187683
Train run 5 loss: 0.7411402463912964
Train run 6 loss: 0.7408127784729004
Train run 7 loss: 0.7403361201286316
Train run 8 loss: 0.7396571040153503
Train run 9 loss: 0.7387100458145142
Train run 10 loss: 0.7374112010002136
Train run 11 loss: 0.7356545925140381
Train run 12 loss: 0.7333085536956787
Train run 13 loss: 0.7302073240280151
Train run 14 loss: 0.7261499166488647
Train run 15 loss: 0.7208762168884277
Train run 16 loss: 0.7140573263168335
Train run 17 loss: 0.7052867412567139
Train run 18 loss: 0.6940373778343201
Train run 19 loss: 0.6796733736991882
Train run 20 loss: 0.6614050269126892
Train run 21 loss: 0.6382409334182739
Train run 22 loss: 0.6090229749679565
Train run 23 loss: 0.5724029541015625
Train run 24 loss: 0.5269359350204468
Train run 25 loss: 0.4711809754371643
Train run 26 loss: 0.4040904641151428
Train run 27 loss: 0.

This confirms model works just fine